Skolemization-Related Explorations
==================

This is a temporary notebook for exploring issues related to a skolemize() method for KnownTruth objects, in particular for known truths involving universal ($\forall$) and existential ($\exists$) quantifiers, with skolemization eventually being part of a more general instantiation or specialization method.

The notebook was generated initially as a copy of the Forall tutorial notebook, and thus while under construction many of the cells and other elements of the original notebook remain.

We import some necessary information then consider an example of a `Forall` object like $\forall_{x \,\in\, S \,|\, Q(x), R(x)} P(x)$:

In [1]:
from proveit import Function, ExprList, Lambda, Literal, singleOrCompositeExpression
from proveit.logic import Forall, Exists, InSet, Equals
from proveit.logic.equality._axioms_ import substitution
from proveit.number import Less, Add, Frac, zero
from proveit._common_ import a, b, c, f, x, y, z, fx, P, Px, Pxy, Q, Qx, R, Rx, Ry, S, t, T
%begin skolemization_explorations

generalExpr =  forall_{A | [not](A)} (A != TRUE)
[_specialized_expression]subbed_expr =  FALSE != TRUE
[_specialized_expression]subbedConditions =  [[not](FALSE)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: FALSE}
AFTER _specialized_expr:
    specializedExpr =  FALSE != TRUE
    requirements =  [[not](FALSE)]
    mappedVarLists =  [(A,), []]
    mappings =  {A: FALSE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  FALSE != TRUE
generalTruth =  |- forall_{A | [not](A)} (A != TRUE)
requirementTruths =  [|- [not](FALSE)]
generalExpr =  forall_{x, y | x != y} (y != x)
[_specialized_expression]subbed_expr =  TRUE != FALSE
[_specialized_expression]subbedConditions =  [FALSE != TRUE]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: FALSE, y: TRU

[_specialized_expression]subbed_expr =  TRUE or [not](TRUE)
[_specialized_expression]subbedConditions =  [TRUE in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: TRUE}
AFTER _specialized_expr:
    specializedExpr =  TRUE or [not](TRUE)
    requirements =  [TRUE in BOOLEANS]
    mappedVarLists =  [(A,), []]
    mappings =  {A: TRUE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  TRUE or [not](TRUE)
generalTruth =  |- forall_{A in BOOLEANS} (A or [not](A))
requirementTruths =  [|- TRUE in BOOLEANS]
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  TRUE
generalTruth =  |- forall_{A in BOOLEANS | A != FALSE} A
requirementTruths =  [|- TRUE in BOOLEANS, |- TRUE != FALSE]
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (x = y) in BOO

[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: _x_, y: _y_}
AFTER _specialized_expr:
    specializedExpr =  (_x_ = _y_) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: _x_, y: _y_}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (_x_ = _y_) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{x, y | x = y} (y = x)
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  ((TRUE = FALSE) = FALSE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: TRUE = FALSE, y: FALSE}
AFTER _specialized_expr:
 

[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: TRUE, y: TRUE}
AFTER _specialized_expr:
    specializedExpr =  (TRUE = TRUE) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: TRUE, y: TRUE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (TRUE = TRUE) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (A = FALSE) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: A, y: FALSE}
AFTER _specialized_expr:
    specializedExpr =  (A = FALSE) in BOOLEANS
    requirement

[_specialized_expression]subbed_expr =  (FALSE = TRUE) or [not](FALSE = TRUE)
[_specialized_expression]subbedConditions =  [(FALSE = TRUE) in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: FALSE = TRUE}
AFTER _specialized_expr:
    specializedExpr =  (FALSE = TRUE) or [not](FALSE = TRUE)
    requirements =  [(FALSE = TRUE) in BOOLEANS]
    mappedVarLists =  [(A,), []]
    mappings =  {A: FALSE = TRUE}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (FALSE = TRUE) or [not](FALSE = TRUE)
generalTruth =  |- forall_{A in BOOLEANS} (A or [not](A))
requirementTruths =  [|- (FALSE = TRUE) in BOOLEANS]
[_specialized_expression]subbed_expr =  [not](FALSE = TRUE)
[_specialized_expression]subbedConditions =  [FALSE != TRUE]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specializ

...

[_specialized_expression]subbed_expr =  (BOOLEANS = {TRUE , FALSE}) or [not](BOOLEANS = {TRUE , FALSE})
[_specialized_expression]subbedConditions =  [(BOOLEANS = {TRUE , FALSE}) in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {A: BOOLEANS = {TRUE , FALSE}}
AFTER _specialized_expr:
    specializedExpr =  (BOOLEANS = {TRUE , FALSE}) or [not](BOOLEANS = {TRUE , FALSE})
    requirements =  [(BOOLEANS = {TRUE , FALSE}) in BOOLEANS]
    mappedVarLists =  [(A,), []]
    mappings =  {A: BOOLEANS = {TRUE , FALSE}}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (BOOLEANS = {TRUE , FALSE}) or [not](BOOLEANS = {TRUE , FALSE})
generalTruth =  |- forall_{A in BOOLEANS} (A or [not](A))
requirementTruths =  [|- (BOOLEANS = {TRUE , FALSE}) in BOOLEANS]
generalExpr =  forall_{x, y | x = y} (y = x)
generalExpr =  forall_{x, y} ((x = y) in BOOLEAN

generalExpr =  forall_{A, B | A supset B} (A supseteq B)
[_specialized_expression]subbed_expr =  Naturals supseteq NaturalsPos
[_specialized_expression]subbedConditions =  [Naturals supset NaturalsPos]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), (B,), []]
[_specialized_expression]mappings =  {A: Naturals, B: NaturalsPos}
AFTER _specialized_expr:
    specializedExpr =  Naturals supseteq NaturalsPos
    requirements =  [Naturals supset NaturalsPos]
    mappedVarLists =  [(A,), (B,), []]
    mappings =  {A: Naturals, B: NaturalsPos}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  Naturals supseteq NaturalsPos
generalTruth =  |- forall_{A, B | A supset B} (A supseteq B)
requirementTruths =  [|- Naturals supset NaturalsPos]
generalExpr =  forall_{A, B | A supset B} (B subset A)
[_specialized_expression]subbed_expr =  NaturalsPos subset Naturals
[_specialized_expression]subbedConditi

[_specialized_expression]mappedVarLists =  [(A,), (B,), []]
[_specialized_expression]mappings =  {A: Reals, B: Naturals}
AFTER _specialized_expr:
    specializedExpr =  Naturals subset Reals
    requirements =  [Reals supset Naturals]
    mappedVarLists =  [(A,), (B,), []]
    mappings =  {A: Reals, B: Naturals}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  Naturals subset Reals
generalTruth =  |- forall_{A, B | A supset B} (B subset A)
requirementTruths =  [|- Reals supset Naturals]
generalExpr =  forall_{A, B | A subset B} (A subseteq B)
[_specialized_expression]subbed_expr =  NaturalsPos subseteq Reals
[_specialized_expression]subbedConditions =  [NaturalsPos subset Reals]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), (B,), []]
[_specialized_expression]mappings =  {A: NaturalsPos, B: Reals}
AFTER _specialized_expr:
    specializedExpr =  NaturalsPos subseteq Reals
    requir

generalExpr =  forall_{A, B | A supset B} (A supseteq B)
[_specialized_expression]subbed_expr =  Complexes supseteq Naturals
[_specialized_expression]subbedConditions =  [Complexes supset Naturals]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), (B,), []]
[_specialized_expression]mappings =  {A: Complexes, B: Naturals}
AFTER _specialized_expr:
    specializedExpr =  Complexes supseteq Naturals
    requirements =  [Complexes supset Naturals]
    mappedVarLists =  [(A,), (B,), []]
    mappings =  {A: Complexes, B: Naturals}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  Complexes supseteq Naturals
generalTruth =  |- forall_{A, B | A supset B} (A supseteq B)
requirementTruths =  [|- Complexes supset Naturals]
generalExpr =  forall_{A, B | A supset B} (B subset A)
[_specialized_expression]subbed_expr =  Naturals subset Complexes
[_specialized_expression]subbedConditions =  [Complexes 

generalExpr =  forall_{x, y | x < y} (y > x)
[_specialized_expression]subbed_expr =  8 > 7
[_specialized_expression]subbedConditions =  [7 < 8]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: 7, y: 8}
AFTER _specialized_expr:
    specializedExpr =  8 > 7
    requirements =  [7 < 8]
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: 7, y: 8}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  8 > 7
generalTruth =  |- forall_{x, y | x < y} (y > x)
requirementTruths =  [|- 7 < 8]
generalExpr =  forall_{x, y | x > y} (y < x)
[_specialized_expression]subbed_expr =  7 < 8
[_specialized_expression]subbedConditions =  [8 > 7]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: 8, y: 7}
AFTER _specialized_expr:
    specializedExpr =  7 < 8
    r

We define some useful literal constants for use elsewhere:

In [2]:
La = Literal('a')
Lb = Literal('b')
Lc = Literal('c')
Ld = Literal('d')
Le = Literal('e')

Le: e

And we define a relatively simple $\forall$ expression, to help investigate and illustrate the distinction between variables and literals, with specialization-to-literals marking the literals as constrained:

In [3]:
basicForallExpr = Forall(x, Px, conditions=[Qx, Rx], domain=S)

basicForallExpr: forall_{x in S | Q(x) , R(x)} P(x)

We can specialize the $\forall$ expression, taking the general $x$ to a specific $a$ (of course in this case, the $x$ and the $a$ are both still general variables, with the ''specialization'' being more conceptual than real)

In [4]:
Qa = Function(Q, a)
Ra = Function(R, a)
basicForallExprSpec01 = basicForallExpr.specialize({x:a}, assumptions=(basicForallExpr, InSet(a, S), Qa, Ra))

generalExpr =  forall_{x in S | Q(x) , R(x)} P(x)
[_specialized_expression]subbed_expr =  P(a)
[_specialized_expression]subbedConditions =  [a in S, Q(a), R(a)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: a}
AFTER _specialized_expr:
    specializedExpr =  P(a)
    requirements =  [a in S, Q(a), R(a)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: a}
assumptionsSet =  frozenset({forall_{x in S | Q(x) , R(x)} P(x)})
assumptions =  [forall_{x in S | Q(x) , R(x)} P(x), a in S, Q(a), R(a)]
specializedTruth assumptions =  (forall_{x in S | Q(x) , R(x)} P(x), a in S, Q(a), R(a))
specializedTruth expr =  P(a)
generalTruth =  {forall_{x in S | Q(x) , R(x)} P(x)} |- forall_{x in S | Q(x) , R(x)} P(x)
requirementTruths =  [{a in S} |- a in S, {Q(a)} |- Q(a), {R(a)} |- R(a)]


basicForallExprSpec01: {forall_{x in S | Q(x) , R(x)} P(x) , a in S , Q(a) , R(a)} |- P(a)

We can instead specialize the $\forall$ expression to a specific literal, taking the general $x$ to the literal constant $c$:

In [5]:
Lc = Literal('c')
Ld = Literal('d')
QLc = Function(Q, Lc)
RLc = Function(R, Lc)
basicForallExprSpec02 = basicForallExpr.specialize({x:Lc}, assumptions=(basicForallExpr, InSet(Lc, S), QLc, RLc))

generalExpr =  forall_{x in S | Q(x) , R(x)} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  [c in S, Q(c), R(c)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  [c in S, Q(c), R(c)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x in S | Q(x) , R(x)} P(x)})
assumptions =  [forall_{x in S | Q(x) , R(x)} P(x), c in S, Q(c), R(c)]
specializedTruth assumptions =  (forall_{x in S | Q(x) , R(x)} P(x), c in S, Q(c), R(c))
specializedTruth expr =  P(c)
generalTruth =  {forall_{x in S | Q(x) , R(x)} P(x)} |- forall_{x in S | Q(x) , R(x)} P(x)
requirementTruths =  [{c in S} |- c in S, {Q(c)} |- Q(c), {R(c)} |- R(c)]


basicForallExprSpec02: {forall_{x in S | Q(x) , R(x)} P(x) , c in S , Q(c) , R(c)} |- P(c)

In [6]:
def testFxn():
    import pdb
    pdb.set_trace()
    return basicForallExpr.usedVars()

The $\forall$ expression now keeps track separately of usedVars and usedLiterals:

In [7]:
basicForallExprSpec02.usedVars()

{P}

In [8]:
basicForallExprSpec02.usedLiterals()

{c}

We can also still get all the literals and variables combined, using a built-in iterator:

In [9]:
list(basicForallExprSpec02.subExprIter())

[P, c]

## Marking Literals as Constrained

Literals now have a ``markAsConstrained()`` method, which will add the attribute ``_constrained`` and set it to True. A Literal is a type of Label, which is a type of Expression, and an Expression is a read-only object -- once created, it's attributes cannot be changed; but we can ADD an attribute and set the value of that attribute. (Interestingly, although we cannot then change the value of that added attribute, we can later simply delete that attribute, thus allowing it to be later recreated and re-set with a possibly different value.)

In [10]:
# clear out any _constrained attribute that might
# have already been set
if hasattr(Lc, '_constrained'):
    delattr(Lc, '_constrained')
hasattr(Lc, '_constrained')

False

In [11]:
# Check if Lc is constrained. Returns False if no _constrained attribute;
# otherwise returns value of the _constrained attribute
Lc.isConstrained()

False

In [12]:
# check to see if Lc has a _constrained attribute
hasattr(Lc, '_constrained')

False

In [13]:
# create _constrained attribute and set it to True
Lc.markAsConstrained()

In [14]:
# now Lc should have the _constrained attribute
hasattr(Lc, '_constrained')

True

In [15]:
# and Lc's _constrained attribute should be True
Lc._constrained

True

In [16]:
# Notice that re-marking as constrained does not elicit an error.
# This is a short-cut allowing us to mark something as constrained
# even if it has already been marked as such
Lc.markAsConstrained()

In [17]:
# Check if Lc is constrained. Returns False if no _constrained attribute;
# otherwise returns value of the _constrained attribute
Lc.isConstrained()

True

In [18]:
# but we can still delete the _constrained attribute, which is a little weird:
if hasattr(Lc, '_constrained'):
    delattr(Lc, '_constrained')
hasattr(Lc, '_constrained')

False

In [19]:
# now calling isConstrained will return false
# even though the attribute doesn't exist:
Lc.isConstrained()

False

## Skolemizing Existential Expressions

We create a handful of various $\exists$ expressions and related KnownTruths:

In [20]:
basicThereExistsExpr00 = Exists(x, Px)

basicThereExistsExpr00: exists_{x} P(x)

In [21]:
basicThereExistsExpr00KT = basicThereExistsExpr00.prove(assumptions={basicThereExistsExpr00})

basicThereExistsExpr00KT: {exists_{x} P(x)} |- exists_{x} P(x)

In [22]:
basicThereExistsExpr01 = Exists(x, Px, conditions=[Qx], domain=S)

basicThereExistsExpr01: exists_{x in S | Q(x)} P(x)

In [23]:
basicThereExistsExpr01KT = basicThereExistsExpr01.prove(assumptions={basicThereExistsExpr01})

basicThereExistsExpr01KT: {exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [24]:
basicThereExistsExpr02 = Exists([x, y], Pxy, conditions=[Qx, Ry], domain=S)

basicThereExistsExpr02: exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [25]:
basicThereExistsExpr02KT = basicThereExistsExpr02.prove(assumptions = {basicThereExistsExpr02} )

basicThereExistsExpr02KT: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [26]:
Txy = Function(T, {x, y})
basicThereExistsExpr03 = Exists([x, y], Pxy, conditions=[Txy, Qx, Ry], domain=S)

basicThereExistsExpr03: exists_{x, y in S | T(x , y) , Q(x) , R(y)} P(x , y)

In [27]:
basicThereExistsExpr03KT = basicThereExistsExpr03.prove(assumptions = {basicThereExistsExpr03})

basicThereExistsExpr03KT: {exists_{x, y in S | T(x , y) , Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | T(x , y) , Q(x) , R(y)} P(x , y)

In [28]:
basicThereExistsExpr03.instanceExpr.conditions

(y in S , T(x , y) , Q(x) , R(y))

In [29]:
(basicThereExistsExpr03.instanceExpr.allConditions()[1]).substituted({z:a})

T(x , y)

In [30]:
Tz = Function(T, z)
Pxyz = Function(P, [x, y, z])
basicThereExistsExpr04 = Exists([x, y, z], Pxyz, conditions=[Qx, Ry, Tz], domain=S)

basicThereExistsExpr04: exists_{x, y, z in S | Q(x) , R(y) , T(z)} P(x , y , z)

In [31]:
Tz = Function(T, z)
Pxyz = Function(P, [x, y, z])
Qxyz = Function(Q, [x, y, z])
basicThereExistsExpr05 = Exists([x, y, z], Pxyz, conditions=[Qxyz], domain=S)

basicThereExistsExpr05: exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [32]:
basicThereExistsExpr05KT = basicThereExistsExpr05.prove(assumptions = {basicThereExistsExpr05} )

basicThereExistsExpr05KT: {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [33]:
Pxyzt = Function(P, [x, y, z, t])
Qxyzt = Function(Q, [x, y, z, t])
basicThereExistsExpr06 = Exists([x, y, z, t], Pxyzt, conditions=[Qxyzt], domain=S)

basicThereExistsExpr06: exists_{x, y, z, t in S | Q(x , y , z , t)} P(x , y , z , t)

In [34]:
basicThereExistsExpr00KT

{exists_{x} P(x)} |- exists_{x} P(x)

In [35]:
QLc = Function(Q, Lc)
basicThereExistsExpr00KTSkolemized = basicThereExistsExpr00KT.skolemize({x:Lc}, assumptions={QLc, InSet(Lc, S)})

ENTERING KT.skolemize()
        processedSubMap so far =  {x: c}
sub is currently:  c
ENTERING KT._skolemized_expressions


basicThereExistsExpr00KTSkolemized: {exists_{x} P(x)} |- P(c)

In [36]:
basicThereExistsExpr00KTSkolemized.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x} P(x)} |- P(c)
1	assumption		{exists_{x} P(x)} |- exists_{x} P(x)

In [37]:
basicThereExistsExpr01KT

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [38]:
Ld = Literal('d')
QLd = Function(Q, Ld)
# A, B, C = (
#     basicThereExistsExpr01KT.skolemize(
#         {x:Ld},
#         assumptions={InSet(Ld, S), QLd}
#     )
# )
basicThereExistsExpr01KTSkolemized = (
    basicThereExistsExpr01KT.skolemize(
        {x:Ld},
        assumptions={InSet(Ld, S), QLd}
    )
)
# skolemizing just one of two possible instance vars
# This is not quite working — somehow subbing into one of the conditions
# but not explicitly then including that condition in the
# subbedConditions list
# basicThereExistsExpr03ThmSkolemized = (
#     basicThereExistsExpr03Thm.skolemize(
#         {y:Ld},
#         assumptions={InSet(Ld, S), RLd}
#     )
# )
A, B, C = basicThereExistsExpr01KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: d}
sub is currently:  d
ENTERING KT._skolemized_expressions


A: {exists_{x in S | Q(x)} P(x)} |- P(d)
B: {exists_{x in S | Q(x)} P(x)} |- d in S
C: {exists_{x in S | Q(x)} P(x)} |- Q(d)

In [39]:
A.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- P(d)
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [40]:
B.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- d in S
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [41]:
C.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- Q(d)
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [42]:
basicThereExistsExpr01KT

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

Very generally when skolemizing, we need to acknowledge free variables.
For example, in the expression $\forall x \exists y [x+y=0]$, we can skolemize y to a constant $y_0$, but need to acknowledge that that constant depends on x, so we might have something like $x + y_0(x) = 0$.
For skolemizing in Prove-It, however, we let the user decide how to specify the Skolem constant(s) and then constrain the resulting expressions later in the context of possible generalizations. For example, if the user has skolemized as discussed above to get the expression $x + y_0(x) = 0$, we could generalize back to obtain $\forall x \exists y [x+y=0]$. If the original skolemization was less explicit, however, producing something like: $x + y_0 = 0$, attempted generalization would simply fail (and the generalization step would have looked for )

This is how Wayne has described it:
$\ldots$ I think I have a better solution that maximizes flexibility and choice for the user.  Let's allow the user to Skolemize either to a simple constant, or a function with a Skolem constant operator and variable operands (we may want to allow literal operands as well).  Now, the rule is simple.  You are not allowed to perform universal generalization if there are any Skolem constants or Skolem functions except Skolem functions that contain the variable being generalized as an argument.  In other words:

1. If your "known truth" contains one or more Skolem constants, you cannot do universal generalization over any variable.

2. If your "known truth" contains any Skolem functions that do not depend upon x, you cannot universally generalize x.

3. If your "known truth" contains only Skolem functions that do depend upon x (e.g., u0(x)), you can universally generalize x.

This allows us to use Skolem functions when we want and simply use Skolem constants when that is just fine for our purposes.

In [43]:
basicThereExistsExpr02KT

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [44]:
innerExists = basicThereExistsExpr02KT.instanceExpr

innerExists: exists_{y in S | R(y)} P(x , y)

In [45]:
innerExists.conditions

(y in S , R(y))

In [46]:
innerExists.instanceExpr.substituted({y:a},)

P(x , a)

In [47]:
innerExists.substituted({y:a},)

exists_{y in S | R(y)} P(x , y)

In [48]:
rebuiltExists = Exists((a,), innerExists.instanceExpr.substituted({y:a},), conditions=[innerExists.conditions.substituted({y:a},)])

rebuiltExists: exists_{a | (a in S , R(a))} P(x , a)

In [49]:
innerExists.conditions[0].freeVars()

{S, y}

In [50]:
basicThereExistsExpr04

exists_{x, y, z in S | Q(x) , R(y) , T(z)} P(x , y , z)

In [51]:
basicThereExistsExpr02KT

{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [52]:
La = Literal('a')              # 'a'
Lb = Literal('b')              # 'b'
Qa = Function(Q, La)           # Q(a)
Rb = Function(R, Lb)           # R(b)
Pab = Function(P, {La, Lb})    # P(a, b)
Ry = Function(R, y)            # R(y)
# basicThereExistsExpr02KTSkolemized = (
#     basicThereExistsExpr02KT.skolemize(
#         {x:La}, assumptions={InSet(La, S), Qa})
# )
basicThereExistsExpr02KTSkolemized = (
    basicThereExistsExpr02KT.skolemize(
        {x:La, y:Lb}, assumptions={})
)
# basicThereExistsExpr02KTSkolemized = (
#     basicThereExistsExpr02KT.skolemize(
#         {y:Lb}, assumptions={})
# )
# basicThereExistsExpr02KTSkolemized
basicThereExistsExpr02KTSkolemized
# D, E, F, G, H = basicThereExistsExpr02KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: a}
        processedSubMap so far =  {x: a, y: b}
sub is currently:  a
sub is currently:  b
ENTERING KT._skolemized_expressions


[{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(a , b),
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- a in S,
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- Q(a),
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- b in S,
 {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- R(b)]

In [53]:
len(basicThereExistsExpr02KTSkolemized)

5

In [54]:
D, E, F, G, H = basicThereExistsExpr02KTSkolemized

D: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(a , b)
E: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- a in S
F: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- Q(a)
G: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- b in S
H: {exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- R(b)

In [55]:
# D, E, F = basicThereExistsExpr02KTSkolemized

In [56]:
D.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- P(a , b)
1	assumption		{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [57]:
E.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- a in S
1	assumption		{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [58]:
F.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- Q(a)
1	assumption		{exists_{x, y in S | Q(x) , R(y)} P(x , y)} |- exists_{x, y in S | Q(x) , R(y)} P(x , y)

In [59]:
basicThereExistsExpr05KT

{exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)

In [60]:
La = Literal('a')              # 'a'
Lb = Literal('b')              # 'b'
Lc = Literal('c')              # 'c'
Qa = Function(Q, La)           # Q(a)
Rb = Function(R, Lb)           # R(b)
Pab = Function(P, {La, Lb})    # P(a, b)
Ry = Function(R, y)            # R(y)

basicThereExistsExpr05KTSkolemized = (
    basicThereExistsExpr05KT.skolemize(
        {x:La, y:Lb}, assumptions={})
)
basicThereExistsExpr05KTSkolemized

ENTERING KT.skolemize()
        processedSubMap so far =  {x: a}
        processedSubMap so far =  {x: a, y: b}
sub is currently:  a
sub is currently:  b
ENTERING KT._skolemized_expressions


[{exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- exists_{z in S | Q(a , b , z)} P(a , b , z),
 {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- a in S,
 {exists_{x, y, z in S | Q(x , y , z)} P(x , y , z)} |- b in S]

In [61]:
QLc = Function(Q, Lc)
RLd = Function(R, Ld)
PLcLd = Function(P, {Lc, Ld})
# Trying to re-use a literal that has already been used as a
# Skolem constant will cause an error. Despite the correct information
# supplied in the skolemize() call below, skolemize() will fail
# and raise an error reporting that one or more of the intended
# Skolem constants has already been used elsewhere.

from proveit import ProofFailure
try:
    basicThereExistsExpr03KT.skolemize({x:Lc, y:Ld}, assumptions={InSet(Lc, S), InSet(Ld, S), QLc, RLd, PLcLd})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

ENTERING KT.skolemize()
        processedSubMap so far =  {x: c}
        processedSubMap so far =  {x: c, y: d}
sub is currently:  c
sub is currently:  d
EXPECTED ERROR: Proof step failed assuming {R(d), d in S, P(c , d), c in S, Q(c), exists_{x, y in S | T(x , y) , Q(x) , R(y)} P(x , y)}: Expecting skolemizeMap substitution values to be Literals not previously used to skolemize. "d" was found to have been marked as having already been used elsewhere to skolemize.


## A brief aside properties/attributes of the Function class

In [62]:
# define a function using a Literal for the function name/symbol
La = Literal('a')
# make Literal 'a' a function of variable y:
LaOfY = Function(La, y)

LaOfY: a(y)

In [63]:
if isinstance(LaOfY, Function):
    print('instance of Function')
    print('operator is: ', LaOfY.operator)
    if isinstance(LaOfY.operator, Literal):
        print('operator is a Literal')
    else:
        print('operator is NOT a literal')
else:
    print('fail') 
print(type(LaOfY))
print(LaOfY.coreInfo())
print(LaOfY.exprInfo())
print(LaOfY.freeVars())
print(LaOfY.operator)
print(type(LaOfY.operator))
singleOrCompositizedVersion = singleOrCompositeExpression(LaOfY)
print(singleOrCompositizedVersion)

instance of Function
operator is:  a
operator is a Literal
<class 'proveit._core_.expression.operation.function.Function'>
('Operation',)
0. a(y)
   core type: Operation
   operator: 1
   operand: 2
1. a
   core type: Literal
   sub-expressions: 
2. y
   core type: Variable
   sub-expressions: 

{y}
a
<class 'proveit._core_.expression.label.literal.Literal'>
a(y)


In [64]:
basicThereExistsExpr00KT

{exists_{x} P(x)} |- exists_{x} P(x)

In [65]:
La = Literal('a')
LaOfb = Function(La, b)
print(LaOfb)
print(type(LaOfb))
print(str(LaOfb.__class__))
basicThereExistsExpr00KT.skolemize({x:LaOfb})

a(b)
<class 'proveit._core_.expression.operation.function.Function'>
<class 'proveit._core_.expression.operation.function.Function'>
ENTERING KT.skolemize()
        processedSubMap so far =  {x: a(b)}
sub is currently:  a
ENTERING KT._skolemized_expressions


{exists_{x} P(x)} |- P(a(b))

In [66]:
basicThereExistsExpr01KT

{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [67]:
La = Literal('a')
LaOfb = Function(La, b)
LaOfx = Function(La, x)
print(LaOfb)
print(type(LaOfb))
print(str(LaOfb.__class__))
basicThereExistsExpr01KTSkolemized = basicThereExistsExpr01KT.skolemize({x:LaOfx})

a(b)
<class 'proveit._core_.expression.operation.function.Function'>
<class 'proveit._core_.expression.operation.function.Function'>
ENTERING KT.skolemize()
        processedSubMap so far =  {x: a(x)}
sub is currently:  a
ENTERING KT._skolemized_expressions


basicThereExistsExpr01KTSkolemized: (P(a(x)) , a(x) in S , Q(a(x)))

In [68]:
A, B, C = basicThereExistsExpr01KTSkolemized

A: {exists_{x in S | Q(x)} P(x)} |- P(a(x))
B: {exists_{x in S | Q(x)} P(x)} |- a(x) in S
C: {exists_{x in S | Q(x)} P(x)} |- Q(a(x))

In [69]:
A.proof()

step type	requirements	statement
0	skolemization	1	{exists_{x in S | Q(x)} P(x)} |- P(a(x))
1	assumption		{exists_{x in S | Q(x)} P(x)} |- exists_{x in S | Q(x)} P(x)

In [70]:
# La = Literal('a')
# LaOfb = Function(La, b)
print(LaOfx)
print(type(LaOfx))
print(str(LaOfx.__class__))

from proveit import ProofFailure
try:
    basicThereExistsExpr00KT.skolemize({x:LaOfx})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

a(x)
<class 'proveit._core_.expression.operation.function.Function'>
<class 'proveit._core_.expression.operation.function.Function'>
ENTERING KT.skolemize()
        processedSubMap so far =  {x: a(x)}
sub is currently:  a
EXPECTED ERROR: Proof step failed assuming {exists_{x} P(x)}: Expecting skolemizeMap substitution values to be Literals not previously used to skolemize. "a" was found to have been marked as having already been used elsewhere to skolemize.


And with a nested $\exists$, we also shouldn't be able to use skolemize multiple instance variables with the same Skolem constant or Skolem function. In the cells below, we refresh the Literal 'a', define a simple nested $exists$ expression, then try to to skolemize both instance variables using the same Literal:

In [71]:
La = Literal('a')
simpleNestedExists = Exists([x, y], Pxy)
simpleNestedExistsKT = simpleNestedExists.prove(assumptions={simpleNestedExists})

simpleNestedExistsKT: {exists_{x, y} P(x , y)} |- exists_{x, y} P(x , y)

In [72]:
from proveit import ProofFailure
try:
    simpleNestedExistsKT.skolemize({x:La, y:La})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

ENTERING KT.skolemize()
        processedSubMap so far =  {x: a}
        processedSubMap so far =  {x: a, y: a}
sub is currently:  a
sub is currently:  a
EXPECTED ERROR: Proof step failed assuming {exists_{x, y} P(x , y)}: Expecting skolemizeMap substitution values to be unique. "a" was found to appear multiple times as a requested skolemization value.


Similarly, we should not be able to skolemize using a Literal that has been previously constrained somewhere else (perhaps in an Axiom or Theorem). In the example below, we artificially constrain a Literal, then try to use it in a skolemization call on the simpleNestedExistsKT example used above:

In [73]:
La = Literal('a')      # refresh the Literal 'a'
Lb = Literal('b')      # refresh the Literal 'b'
Lb.markAsConstrained() # mark 'b' as constrained
from proveit import ProofFailure
try:
    simpleNestedExistsKT.skolemize({x:La, y:Lb})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

ENTERING KT.skolemize()
        processedSubMap so far =  {x: a}
        processedSubMap so far =  {x: a, y: b}
sub is currently:  a
sub is currently:  b
EXPECTED ERROR: Proof step failed assuming {exists_{x, y} P(x , y)}: Expecting skolemizeMap substitution values to be Literals not previously constrained. "b" was found to have been previously constrained elsewhere (perhaps in an Axiom or Theorem).


Attempting to skolemize a Forall expression should fail and produce an error message, alerting the user to the fact that only an Exists expression can be skolemized:

In [74]:
from proveit import ProofFailure
try:
    substitution.skolemize({x:Lc}, assumptions={Qa})
    assert False, "Expecting a Skolemization error; should not make it to this point."
except ProofFailure as e:
    print("EXPECTED ERROR:", e)

ENTERING KT.skolemize()
        processedSubMap so far =  {x: c}
EXPECTED ERROR: Proof step failed assuming {Q(a)}: May only skolemize an Exists expression.


But right now, we can still specialize a Forall $\forall$ KnownTruth using a Literal that has been previously used as a Skolem constant, and this makes sense because a $\forall$ is indeed “for all”:

In [75]:
substitutionSpec = substitution.specialize({x:Lc, y:Ld}, assumptions=[Equals(Lc, Ld)])

generalExpr =  forall_{x, y} ((x = y) in BOOLEANS)
[_specialized_expression]subbed_expr =  (c = d) in BOOLEANS
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: c, y: d}
AFTER _specialized_expr:
    specializedExpr =  (c = d) in BOOLEANS
    requirements =  []
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: c, y: d}
assumptionsSet =  frozenset()
assumptions =  []
specializedTruth assumptions =  ()
specializedTruth expr =  (c = d) in BOOLEANS
generalTruth =  |- forall_{x, y} ((x = y) in BOOLEANS)
requirementTruths =  []
generalExpr =  forall_{A in BOOLEANS} (A or [not](A))
[_specialized_expression]subbed_expr =  (c = d) or [not](c = d)
[_specialized_expression]subbedConditions =  [(c = d) in BOOLEANS]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(A,), []]
[_specialized_expression]mappings =  {

requirementTruths =  []
[_specialized_expression]subbed_expr =  f(c) = f(d)
[_specialized_expression]subbedConditions =  [f(d) = f(c)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), (y,), []]
[_specialized_expression]mappings =  {x: f(d), y: f(c)}
AFTER _specialized_expr:
    specializedExpr =  f(c) = f(d)
    requirements =  [f(d) = f(c)]
    mappedVarLists =  [(x,), (y,), []]
    mappings =  {x: f(d), y: f(c)}
assumptionsSet =  frozenset()
assumptions =  [c = d]
specializedTruth assumptions =  (c = d,)
specializedTruth expr =  f(c) = f(d)
generalTruth =  |- forall_{x, y | x = y} (y = x)
requirementTruths =  [{c = d} |- f(d) = f(c)]


substitutionSpec: {c = d} |- f(c) = f(d)

## Working to understand more carefully what happens in the instantiation of a simple Forall expression using ``specialize()``, so we can better understand how to implement the skolemize.

In [76]:
basicForAllExpr = Forall(x, Px)

basicForAllExpr: forall_{x} P(x)

In [77]:
basicForAllExpr

forall_{x} P(x)

In [78]:
# from proveit._core_ import KnownTruth
# tempKT = KnownTruth(basicForAllExpr, {basicForAllExpr})

In [79]:
# print(tempKT)

In [80]:
theOperand = basicForAllExpr.operand

theOperand: x -> P(x)

In [81]:
theOperand.parameterVars

(x,)

In [82]:
theOperand.body

P(x)

In [83]:
theOperand.conditions

()

In [84]:
basicForAllExprThm = basicForAllExpr.prove(assumptions = {basicForAllExpr})

basicForAllExprThm: {forall_{x} P(x)} |- forall_{x} P(x)

In [85]:
basicForAllExprThm.exprInfo()

,core type,sub-expressions,expression
0,Operation,operator: 1operand: 2,
1,Literal,,
2,Lambda,parameter: 5body: 3,
3,Operation,operator: 4operand: 5,
4,Variable,,
5,Variable,,


In [86]:
basicForAllExprThm.specialize({x:Lc})

generalExpr =  forall_{x} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  []
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x} P(x)})
assumptions =  [forall_{x} P(x)]
specializedTruth assumptions =  (forall_{x} P(x),)
specializedTruth expr =  P(c)
generalTruth =  {forall_{x} P(x)} |- forall_{x} P(x)
requirementTruths =  []


{forall_{x} P(x)} |- P(c)

In [87]:
basicForAllExprThm.specialize({x:Lc}).proof()

generalExpr =  forall_{x} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  []
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  []
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x} P(x)})
assumptions =  [forall_{x} P(x)]
specializedTruth assumptions =  (forall_{x} P(x),)
specializedTruth expr =  P(c)
generalTruth =  {forall_{x} P(x)} |- forall_{x} P(x)
requirementTruths =  []


step type	requirements	statement
0	specialization	1	{forall_{x} P(x)} |- P(c)
	x : c
1	assumption		{forall_{x} P(x)} |- forall_{x} P(x)

In [88]:
basicForAllExpr02 = Forall(x, Px, conditions={Qx})

basicForAllExpr02: forall_{x | Q(x)} P(x)

In [89]:
basicForAllExpr02Thm = basicForAllExpr02.prove(assumptions={basicForAllExpr02})

basicForAllExpr02Thm: {forall_{x | Q(x)} P(x)} |- forall_{x | Q(x)} P(x)

In [90]:
basicForAllExpr02ThmSpec = basicForAllExpr02Thm.specialize({x:Lc}, assumptions={QLc})

generalExpr =  forall_{x | Q(x)} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  [Q(c)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  [Q(c)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}
assumptionsSet =  frozenset({forall_{x | Q(x)} P(x)})
assumptions =  [Q(c), forall_{x | Q(x)} P(x)]
specializedTruth assumptions =  (Q(c), forall_{x | Q(x)} P(x))
specializedTruth expr =  P(c)
generalTruth =  {forall_{x | Q(x)} P(x)} |- forall_{x | Q(x)} P(x)
requirementTruths =  [{Q(c)} |- Q(c)]


basicForAllExpr02ThmSpec: {Q(c) , forall_{x | Q(x)} P(x)} |- P(c)

In [91]:
basicForAllExpr02ThmSpec = basicForAllExpr02Thm.specialize({x:Lc})

generalExpr =  forall_{x | Q(x)} P(x)
[_specialized_expression]subbed_expr =  P(c)
[_specialized_expression]subbedConditions =  [Q(c)]
[_specialized_expression]requirements =  []
[_specialized_expression]mappedVarLists =  [(x,), []]
[_specialized_expression]mappings =  {x: c}
AFTER _specialized_expr:
    specializedExpr =  P(c)
    requirements =  [Q(c)]
    mappedVarLists =  [(x,), []]
    mappings =  {x: c}


SpecializationFailure: Unable to prove P(c) assuming {forall_{x | Q(x)} P(x)}: Unmet specialization requirement: Q(c)

In [ ]:
basicForAllExpr02ThmSpec.asImplication(basicForAllExpr02ThmSpec.assumptions)

In [ ]:
%end skolemization_explorations